<a href="https://colab.research.google.com/github/Joshua-Pow/pokescanner/blob/main/PokeScanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Importing libraries / Data Loading and Splitting



In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt

from torchvision import datasets, models, transforms

use_cuda = True

torch.manual_seed(1) # set the random seed

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) #Must have folder inside drive called PokeScanner

data_transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.Resize(250),
    transforms.ToTensor()
])

Mounted at /content/gdrive


In [ ]:
filePath = '/content/gdrive/MyDrive/PokeScanner' #Everyone has had this folder shared with them, Right click the folder and click "Add shortcut to Drive"
train_set = datasets.ImageFolder(filePath+'/training', transform=transform) #NEED TO SPLIT DATA
validation_set = datasets.ImageFolder(filePath+'/validation', transform=transform)
testing_set = datasets.ImageFolder(filePath+'/testing', transform=transform)

print('Number of Images:', len(train_set))

Number of Images: 6820


### 2. Model Building and Sanity Checking

Build a convolutional neural network model that takes the (224x224 RGB) image as input, and predicts the pokemon

In [ ]:
class PokemonClassifier(nn.Module):
      def __init__(self, kernel_size):
          super(PokemonClassifier, self).__init__()
          self.conv1 = nn.Conv2d(3, 2, kernel_size) #in_channels, out_chanels, kernel_size
          self.pool = nn.MaxPool2d(2, 2) #kernel_size, stride 
          self.conv2 = nn.Conv2d(2, 2, kernel_size) #in_channels, out_chanels, kernel_size
          self.fc1 = nn.Linear(5618, 32) #needs input-dependent value
          self.fc2 = nn.Linear(32, 9)

      def forward(self, x):
          x = self.pool(F.relu(self.conv1(x)))
          x = self.pool(F.relu(self.conv2(x)))
          x = x.view(-1, 5618)
          x = F.relu(self.fc1(x))
          x = self.fc2(x)
          return x

In [ ]:
#From tut3a
def get_accuracy(model, accuracy_data, batch_size):
    correct = 0
    total = 0
    for imgs, labels in torch.utils.data.DataLoader(accuracy_data, batch_size=batch_size):
        
        
        #############################################
        #To Enable GPU Usage
        if use_cuda and torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()
        else:
          print("get_accuracy: no cuda")
        #############################################
        
        
        output = model(imgs)
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [ ]:
#From tut3a
def train(model, data, size, num_epochs, learning_rate):
    train_loader = torch.utils.data.DataLoader(data, batch_size=size, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    iters, losses, train_acc, val_acc, epochList = [], [], [], [], []

    # training
    n = 0 # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(train_loader):

            #############################################
            #To Enable GPU Usage
            if use_cuda and torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()
            else:
              print("Train: no cuda")
            #############################################
              
            out = model(imgs)             # forward pass

            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch

            # save the current training information
            iters.append(n)
            losses.append(float(loss)/size)             # compute *average* loss
            n += 1
        train_acc.append(get_accuracy(model, accuracy_data=train_set, batch_size=size)) # compute training accuracy 
        val_acc.append(get_accuracy(model, accuracy_data=validation_set, batch_size=size))  # compute validation accuracy
        epochList.append(epoch)
        print("Epoch {}: Training Accuracy: {} Validation Accuracy: {}".format(epoch, train_acc[epoch], val_acc[epoch]))
    
    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(epochList, train_acc, label="Train")
    plt.plot(epochList, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
  


In [ ]:
model = PokemonClassifier()

#proper model
#############################################
#To Enable GPU Usage
if use_cuda and torch.cuda.is_available():
  model.cuda()
  print("cuda")
#############################################

#Basic first attempt at training
train(model, train_set, size=64, num_epochs=10, learning_rate=0.001)

In [ ]:
#Hello World
#Test Line